# GAN - Генеративно-состязательные нейронные сети

### TOC
* Введение
    * Задача генерации
    * Latent space
    * Наивный подход


* GAN - Генеративно-состязательная сеть (англ. Generative adversarial network, сокращённо GAN)
    * Дискриминатор
    * GAN - принципы работы
    * Пример генерации точек на параболе


* DCGAN (Deep Convolutional Generative Adversarial Nets)
    * DCGAN - GAN для генерации изображений
    * deconv - обратные свёртки
    * Пример генерации изборажений


* cGAN - Условные порождающие состязательные сети (Conditional Generative Adversarial Nets)
    * Принципы работы cGAN
    * Модификации cGAN


* GAN Zoo
    * ProGAN - метод обучения сетей до высокого разрешения
    * Domain Transfer Network - перенос стиля с помомщью GAN
    * SRGAN и StackGANs - сети для увеличения разрешения
    * pix2pix - генерация изображения по его рисунку или из сегментации
    * Семантическая генерация - изображение по входному вектору из целевых свойств
    * ControlGAN и Text to Image - генерация изображения по текстовому описанию

* Заключение

* Ссылки

## Введение

Когда мы изучаем окружающий мир, то у нас нет ответов что представляет из себя окружающая действительность, однако, со временем, мы учимся распознавать образы, речь, движения, благодаря тому, что мы просто смотрим и слушаем! Если детям сказать "Это собака" - это ничтожно маленькая информация. 

Положим, если давать человеку бит в секунду, и, учитывая что человек живёт в среднем $10^9$ секунд, а так же то, что в мозге человека примерно $10^{14}$ нейронных связей, значит человек использует мозг на одну стотысячную. Откуда ему получить столько информации чтобы стать человеком? Изучать мир самому! Анализируя все входные источники своего тела: глаза, уши и так далее.

Классы, к которым мы привыкли в курсе нейронных сетей, в изображении, которое воспринимают наши глаза отсутствуют. То есть преимущественно мы учимся без учителя. Однако мы способны к творчеству, мы можем писать картины, музыку. Придумывать и прогнозировать. Мы можем генерировать что-то новое.

Как подойти к такой задаче с помощью нейронных сетей?

## Поставим задачу генерации.

Мы имеем неразмеченные данные, и хотим генерировать новые данные, которые будут удовлетворять следующим условиям:
* Новые данные должны быть похожи на исходные
* При этом не повторять их в точности (или повторять, при случайном стечении обстоятельств)

### Эволюция в генерации изображений лиц:

**https://thispersondoesnotexist.com**

<img src="face_evol.jpg" alt="alttext" style="width: 700px;"/>

## Latent space

Поскольку мы обусловились о случайности, то нам эту случайность нужно реализовать. Как это сделать с нейронной сетью? В сетях, которые мы проходили при подаче экземпляра и фиксированных весах всегда получается один и тот же результат. Случайно изменять веса сети? Есть способ лучше - давать в качестве входа случайных шум.

![random_example.png](random_example.png)

Почему именно вектор? Почему не одно случайное число. Ответ прост: изменяя одно случайное число, мы будем перемещаться на очень небольшое расстояние и генерация будет однообразной. Поскольку мы физически не можем генерировать числа в бесконечных пределах, мы вынуждены использовать фиксированный диапазон.

В одномерном случае, взяв диапазон [0,1] у нас всего одна размерность нашего перемещения.
Если взять двухмерный, или трёхмерный случай, то у нас уже есть две и три **степени свободы**.

То есть случайный шум большой размерности даёт нам больше объёма для генерации. Это называется **input latent space** - входное латентное простанство.

Note: из-за неустоявшейся терминологии, случайное распределение на входе генератора называется латентным пространством, так же как и скрытое пространство в автоэнкодерах. Поэтому в этой лекции будем называть его **входным** латентным пространством. Так же встречается вариант: predefined latent space.

#### Какую размерность и форму выбрать?

Длина вектора (размерность латентного пространства) выбирается больше, чем количество разных
независимых свойств объекта, которые мы хотим получить. Если длина 0, то случайности
нет и генератор будет всегда производить один и тот же объект. Если длина 1, то будет
шкала, вдоль которой будут расположены, например, генерируемые изображения. Для генератора
лиц, это будет, в лучшем случае, шкала от молодой женщины блондинки к пожилому мужчине
брюнету. Лучший способ выбрать длину вектора - это посмотреть похожую задачу в публикациях,
взять подобную размерность, и начать экспериментировать с размерностью оттуда.

У большого латентного пространства есть минусы: увеличивая размерность латентного пространства мы можем расширить его настолько, что при обучении модели точек в этом латентром пространстве будет настолько мало, что в основном пространство будет состоять из пустот. Тогда модель будет крайне некачественно "понимать" что ей нужно генерировать в точке, которой не было на обучающей выборке.

Аналогичный вопрос встаёт и по поводу формы распределения входного латентного пространства. Как мы знаем из лекции про обучение сети, инициализация весов и нормализация имеют существенный вклад в работу модели. Поэтому, принято использовать многомерное нормальное распределение для input latent space, потому что оно лучше взаимодействует с весами модели и увеличивает сходимость.

<img src ="MultivariateNormal.png" width="600">

## Наивный подход
(как делать на практике не нужно)

Самым тривиальным решением кажется подача случайного шума на вход и реальные точки параболы будем ожидать на выходе. Проверим как это будет работать.

In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data.dataset import Dataset, random_split
from sklearn.utils import shuffle
from IPython.display import clear_output
import matplotlib.pyplot as plt
import torchvision

In [ ]:
def gen_pair(num=100):
    x = np.random.uniform(low=-1, high=1, size=(num,))
    y = x*x
    return np.hstack((x.reshape(-1,1), y.reshape(-1,1))) # Создадим num корректных точек (x,y) на параболе

pairs = gen_pair(100)
plt.scatter(pairs[:,0], pairs[:,1])
plt.title("Случайные точки на параболе, которые используем в качестве датасета.")
plt.show()

Объединим наши точки на параболе со случайным входным шумом.

In [ ]:
n_batches = 10
batch_size = 128
ls = 3 # latent space

noice = np.random.normal(size=(n_batches*batch_size, ls))
noice = torch.tensor(noice, dtype=torch.float)
print(f"noice.shape: {noice.shape}")

xy_pair = gen_pair(num = n_batches*batch_size)
xy_pair = torch.tensor(xy_pair, dtype=torch.float)
print(f"xy_pair.shape: {xy_pair.shape}")

dataset = TensorDataset(noice, xy_pair) # model inputs, model outputs

len_tr = int(len(dataset)*0.8)
len_tst = len(dataset) - len_tr
trainset, testset = random_split(dataset,[len_tr,len_tst])

train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(testset, batch_size=batch_size, shuffle=True)

In [ ]:
def get_test_loss(model,test_loader,loss_function):
    with torch.no_grad():
        loss_test_total = 0
        for samples, labels in test_loader:
            outputs = model(samples.cuda())
            loss = loss_function(outputs, labels.cuda())
            loss_test_total += loss.item()
        return loss_test_total/len(test_loader)

Определим простую модель, которая будет ожидать шум на вход, и генерировать точки на выходе. (Обратите внимание, что функция активации на последнем слое отсутствует, поскольку мы не ограничиваем наш генератор в каком-то диапазоне.)

In [ ]:
class GenModel(nn.Module):
    def __init__(self, latent_space):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_space, 50),
            nn.ReLU(),
            nn.Linear(50, 50),
            nn.ReLU(),
            nn.Linear(50,2))

    def forward(self, x):
        return self.model(x)

In [ ]:
epochs = 500
model = GenModel(latent_space = ls)
model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_function = nn.L1Loss().cuda()

for epoch in range(epochs):
    loss_epoch = 0
    for samples, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(samples.cuda())
        loss = loss_function(outputs.cuda(), labels.cuda())
        loss.backward()
        optimizer.step()
        loss_epoch += loss.item()
        
    loss_test = get_test_loss(model,test_loader,loss_function)
    if epoch%100==0:
        print(f"Epoch={epoch} train_loss={loss_epoch/len(train_loader):.4} test_loss={loss_test:.4}")
        # test_image(model)

Посмотрим результаты генерации на шуме

In [ ]:
def test_image(model):
    model.cpu()
    noice = torch.tensor(np.random.normal(size=(1000, ls)), dtype=torch.float)
    xy_pair_gen = model(noice)

    xy_pair_gen = xy_pair_gen.detach().numpy()
    plt.scatter(xy_pair_gen[:,0], xy_pair_gen[:,1])
    plt.axis([-1, 1, 0, 1])
    plt.show()
    model.cuda()
test_image(model)

### Причина неудачи

Очевидно, что модель обучается плохо. Давайте разберёмся почему. В одном сэмпле данных мы подаём конкретный шум и конкретную точку, в следующем сэмпле так же. Поскольку в качестве лосс-функции была выбрана L1Loss, то генерация модели по батчу будет устремлять градиент в усреднённую позицию, которая не будет соответствовать позиции точки на параболе. (Хотя в целом, это не нереально)

**Нужно придумать как сделать так, чтобы точки "растолкать" из области, где парабола отсутствует.**

## Дискриминатор

Чтобы решить проблему сильного разброса точек (оттолкнуть их от пустот) можно наказывать нейронную сеть не напрямую лосс функцией, а сетью, которая будет говорить нам что точка лежит на параболе или не лежит. Однако, проверять это условие анализируя пару x, y на принадлежность к параболе плохая идея. Потому что точки получаются из случайного шума и ожидать что они попадут точно в параболу мы не имеем права. Поскольку попадание должно быть выше чем точность типа данных (32-bit floating point для torch.float). Мы рискуем не получить в процессе обучения ни одной точки, попавшей в параболу. Нам нужно дейстовать мягче.

Поэтому, определим сеть-классификатор точек (лежит/не лежит), которую назовём **дискриминатор или критик**.

In [ ]:
class DisModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(2, 25),
            nn.ReLU(),
            nn.Linear(25, 15),
            nn.ReLU(),
            nn.Linear(15,1),
            nn.Sigmoid())
        
    def forward(self, x):
        return self.model(x)

Итого мы имеем: генератор, выдающий точки то ли попадая в параболу, то ли не попадая в неё. А так же дискриминатор, который будет учиться их различать. Поэтому мы будем подавать на генератор синтетические правильные точки, считая их реальными, и точки, которые выдаёт генератор, считая их подделкой.

Таким образом, генератор будет учиться подражать реальным данным, а дискриминатор будет учиться отличать реальные точки, от подделок. 

Мы пришли к идее генеративно-состязательных нейронных сетей.

##  Generative adversarial network (GAN)

G - генератор

D - дискриминатор

<img src ="http://edunet.kea.su/repo/src/L13_GAN_cGAN/img/11-15.png" width="700">

Математически - это игра двух игроков:

<img src ="http://edunet.kea.su/repo/src/L13_GAN_cGAN/img/11-16.png" width="700">

Чёрным $P_x$ обозначено реальное распределение сэмплов, зелёным $P_g$ - результаты генерации генератора, а синим - разделение между реальными и сгенерированными образцами. В идеальном случае справа дискриминатор перестаёт различать реальные объекты и сгенерированные.

<img src ="http://edunet.kea.su/repo/src/L13_GAN_cGAN/img/11-17.png" width="500">

#### Теоретическая гарантия сходимости

<img src ="http://edunet.kea.su/repo/src/L13_GAN_cGAN/img/11-18.png" width="500">

<img src ="http://edunet.kea.su/repo/src/L13_GAN_cGAN/img/11-19.png" width="500">

#### Математическое описание процесса обучения

<img src ="http://edunet.kea.su/repo/src/L13_GAN_cGAN/img/11-20.png" width="500">

https://arxiv.org/pdf/1701.07875.pdf

В процессе совместного конкурентного обучения, если система достаточно сбалансирована, достигается минимаксное состояние равновесия, в котором обе сети значительно улучшили своё качество, и теперь сгенерированные изображения могут быть использованы практически как настоящие.

**Преимущества GAN**
* Теоретические гарантии сходимости
* Можно обучать обычным SGD
* Решает в явном виде задачу generative modeling
* Но неявным образом (нейросети)


**Недостатки GAN**
* Нестабильное обучение
* Очень долгая сходимость
* Mode-collapsing
* Generator/Discriminator starvation
* Поиск оптимальных параметров - pure luck

### GAN Практический пример

Создадим две функции возвращающие батчи:

Первая функция будет создавать случайный вектор на вход генератора и метками классов будут единицы, потому что мы хотим чтобы генератор давал такие точки, чтобы классификатор их распознавал как единицы (верные). Поэтому лосс функция будет подгонять веса генератора к единицам.

Вторая функция будет давать батч на вход дискриминатора, смешивая:
* Истиные точки на параболе (чтобы обучать дискриминатор)
* Точки не на параболе (чтобы дискриминатор знал где "не парабола")
* Выход генератора как неверный (чтобы генератор не коллапсировал в одну верную точку)

Во второй функции можно закомментить разные типы точек (не забыв убрать их из z = torch.vstack) и посмотреть на результаты. Например, если убрать gan_pair, то генератор будет коллапсировать в одну точку.

In [ ]:
def get_g_batch(batch_size, latent_dim):
    g_input = torch.randn(size=(batch_size, latent_dim)).cuda()
    labels = torch.ones(size=(batch_size,)).cuda()
    return g_input, labels

def get_real_mix_batch(batch_size, latent_dim, netG):
    # Generate true pairs and true labels
    x = torch.distributions.Uniform(-1, +1).sample((batch_size,)).cuda()
    true_pair = torch.vstack((x, x*x)).T.cuda()
    true_labels = torch.ones(size=(batch_size,)).unsqueeze(1).cuda()
    
    # Generate fake uniform pairs and fake labels
    x_fake = torch.distributions.Uniform(-1, +1).sample((batch_size,)).cuda()
    y_fake = torch.distributions.Uniform(-1, +1).sample((batch_size,)).cuda()
    fake_pair = torch.vstack((x_fake, y_fake)).T
    fake_labels = torch.zeros(size=(batch_size,)).unsqueeze(1).cuda()

    # Generate points from generator and set labels as fake
    gan_pair = netG(torch.randn(size=(batch_size, latent_dim)).cuda())
    gan_labels = torch.zeros(size=(batch_size,)).unsqueeze(1).cuda()
    
    # Stack all types of points
    z = torch.vstack((torch.hstack((true_pair, true_labels)),
                      torch.hstack((fake_pair, fake_labels)),
                      torch.hstack((gan_pair, gan_labels))
                     ))
    # Shuffle
    z=z[torch.randperm(z.size()[0])]
    
    # Split back to samples and labels
    mixed_pairs = z[:, :2]
    mixed_labels = z[:, 2]
    return mixed_pairs, mixed_labels

Для удобства обернём в функции шаги backpropagation дискриминатора и генератора

In [ ]:
def netD_step(netD, batchD, loss_func, optimizer):
    samples, labels = batchD
    optimizer.zero_grad()
    outputs = netD(samples.cuda())
    loss = loss_func(outputs.cuda(), labels.unsqueeze(1).detach().cuda())
    loss.backward()
    optimizer.step()

In [ ]:
def netG_step(netD, netG, batchG, loss_func, optimizer):
    samples, labels = batchG
    optimizer.zero_grad()
    outputs = netD(netG(samples.cuda()))
    loss = loss_func(outputs.cuda(), labels.unsqueeze(1).detach().cuda())
    loss.backward()
    optimizer.step()

Будем каждую эпоху отображать что уже умеет генерировать генератор

In [ ]:
def plot_gen(netG, epoch="Not provided"):
    Gin, _ = get_g_batch(1000, latent_dim)
    out = netG(Gin).cpu()
    plt.scatter(out.detach().numpy()[:, 0], out.detach().numpy()[:, 1], color="blue", s=1)
    plt.title(f'Generator points. End of epoch= {epoch+1}', fontsize=10)
    plt.axis([-1,1,-0.5,1])
    plt.show()

Поскольку сети обучаются по очереди, то создадим два оптимайзера.

In [ ]:
latent_dim = 5
batch_size = 128
batch_per_epoch = 2000
epochs = 5

netG = GenModel(latent_dim).cuda()
netD = DisModel().cuda()
loss_func = nn.BCELoss().cuda()
optD = torch.optim.Adam(netD.parameters(), lr=0.001)
optG = torch.optim.Adam(netG.parameters(), lr=0.001)

В цикле обучения будет создаваться батч данных (размера batch_size) для генератора, и батч из смеси точек (размера 3\*batch_size) для дискриминатора. С балансом данных подаваемых на обе сети можно эксперементировать.

In [ ]:
def train(netD, netG, batch_per_epoch, batch_size, latent_dim, epochs, loss_func, optD, optG):
    for epoch in range(epochs):
        for _ in range(batch_per_epoch):
            batchG = get_g_batch(batch_size, latent_dim)
            batchD = get_real_mix_batch(batch_size, latent_dim, netG)
            
            netD.train(True)
            netG.train(False)
            netD_step(netD, batchD, loss_func, optD)
            
            netD.train(False)
            netG.train(True)
            netG_step(netD, netG, batchG, loss_func, optG)
            
        # clear_output()
        plot_gen(netG, epoch)

In [ ]:
train(netD, netG, batch_per_epoch, batch_size, latent_dim, epochs, loss_func, optD, optG)

Сеть отлично обучилась при подаче в дискриминатор всех типов точек. Можно отключить некоторые типы и посмотреть что получится.

## DCGAN - Генерация изображений

С помощью GAN можно, разумеется, генерировать не только точки на параболе. Можно генерировать, например, изображения. Но поднимаются закономерные вопросы.

### Как из шума на входе сети получить изображение?

Самым простым ответом будет: взять шум, пропустить его через полносвязные слои и сделать rashape до нужного разрешения. В целом, это будет работать.


Однако **DCGAN - Deep Convolutional GAN** использует сверточные и сверточно-транспонированные (convolutional and convolutional-transpose) слои в дискриминаторе и генераторе соответственно. Впервые метод DCGAN был описан Рэдфордом и др. в статье "Обучение неконтролируемому представлению с помощью Глубоких Сверточных генеративных состязательных сетей".

<img src ="dcgan_arcit.jpg" width="700">

Ниже видна разница в генерации при помощи исключительно полносвязных слоёв и при помощи обратных свёрток. Очевидно, результат DCGAN лучше чем GAN.

<img src ="gan_dcgan.png" width="700">

#### Архитектура DCGAN

На вход порождающей сети поступает шум, который по факту представляет из себя вектор произвольной размерности. Отсюда следует, что нет никакой необходимости расчленять его на признаки с помощью сверточных слоев в отличие от дискриминатора, а требуется, наоборот, преобразовать в результирующее изображение. Для выполнения этой задачи будут использоваться так называемые разверточные (deconvolutional) слои, которые по факту являются обратными к сверточным. Принцип их действия заключается в том, что они дополняют вход нулями до матрицы необходимого размера.

<img src ="dcgan.PNG" width="700">

#### Как работают обратные свёртки?

Обратные свёртки/Развёртки (deconvolutional, convolutional-transpose). Входное разрешение дополняется нулями (подобно паддингу) до такого разрешения, чтобы выходное разрешение было больше входного. Вы можете возразить: но ведь сильно увеличить изображение не получится! Для этого слоёв ставится несколько, чтобы каждый новый слой лучше восстанавливал то, что в него подают. Итеративно увеличивая разрешения, ядра свёрток учатся генерировать изображения.

<img src ="tconv_1.JPG" width="600">

<img src ="tconv_1_padding.JPG" width="600">

[nn.ConvTranspose2d](https://pytorch.org/docs/stable/generated/torch.nn.ConvTranspose2d.html#torch.nn.ConvTranspose2d)


In [ ]:
x = torch.ones((1, 3, 10, 10)) # Like one 3-channal image with 10x10 size

In [ ]:
convT = nn.ConvTranspose2d(3, 8, kernel_size=3)
y = convT(x)

In [ ]:
y.shape # One 8-chanells image with 12x12 size

<img src ="convtransponce_doc.jpg" width="700">

## Пример обученного DCGAN

In [ ]:
use_gpu = True if torch.cuda.is_available() else False

model = torch.hub.load('facebookresearch/pytorch_GAN_zoo:hub', 'DCGAN', pretrained=True, useGPU=use_gpu)

In [ ]:
num_images = 16
noise, _ = model.buildNoiseData(num_images)
with torch.no_grad():
    generated_images = model.test(noise)

fig, ax = plt.subplots(figsize=(16*3, 2*3))
ax.imshow(torchvision.utils.make_grid(generated_images).permute(1, 2, 0).cpu().numpy(), interpolation='nearest', aspect='equal')
plt.show()

## cGAN - GAN с условием

cGAN расшифровывается как Conditional Generative Adversarial Net - это GAN с условием. Условие может быть любым, например, генерация конкретной цифры. В этом случае нам нужен уже размеченный датасет, для того чтобы обучить дискриминатор.

<img src ="cGAN_switcher.JPG" width="400">

Уловием в данном случае будет label, который на рисунке обозначен **Y**. Label добавляется к случайному шуму, тем самым мы говорим генератору генерировать случайное изображение нужного класса. Так же он подаётся в дискриминатор в качестве входа, чтобы дискриминатор знал какое изображение классифицировать как реальное, а какое как вымышленное.

Обучение в данном случае будет аналогичным обучению GAN, мы будем обучать сети, чередуя реальные данные и сгенерированные, добавив label.

<img src ="cGANS_results.JPG" width="700">

#### Модификации cGAN

Метки классов можно подавать не только способом, описанным выше. Можно вместо подачи их в дискриминатор сделать так, чтобы он их предсказывал - **Semi-Supervized GAN**.

Или же не подавать label в дискриминатор, но ждать от него классификации в соответствии с классом, который мы хотим получить от генератора - это **InfoGAN**

Ещё одна модификация cGAN - это **AC-GAN** (auxiliary classifier) в которой единственное различие заключается в том, что дискриминатор дожен помимо распознавания реальных и фейковых изображений ещё и классифицировать их. Он имеет эффект стабилизации процесса обучения и позволяет генерировать большие высококачественные изображения, изучая представление в скрытом пространстве, которое не зависит от метки класса.

<img src ="gans_zoo.jpg" width="800">

# GAN Zoo

GAN моделей настолько много, что нет смысла рассказывать о всех. Так или иначе, сейчас многие новые нейросети используют принципы GAN для обучения. Будь-то распознавание или сегментация.

https://paperswithcode.com/methods/category/generative-models

### ProGAN

https://arxiv.org/pdf/1710.10196.pdf

"PROGRESSIVE GROWING OF GANS FOR IMPROVED QUALITY, STABILITY, AND VARIATION" - метод, призванный ускорить обучение сети генерации изображений с помощью добавления слоёв во время генерации. Сначала сеть учится генерировать изображения низкого разрешения, потом добавляется слой, который создаёт более высокое разрешение из фич предыдушего слоя. Эта идея позволяет плавно обучать сеть и добиваться лучшей сходимости.

<img src ="ProGAN.gif" width="700">

<img src ="http://edunet.kea.su/repo/src/L13_GAN_cGAN/img/11-23.gif" width="700">

### Domain transfer network

https://arxiv.org/abs/1611.02200

Domain transfer network (DTN)- сеть, совмещающая в себе автоэнкодер, и gan для задачи переноса стиля.

<img src ="http://edunet.kea.su/repo/src/L13_GAN_cGAN/img/11-27.png" width="600">

#### Результаты

From SVHN to MNIST

<img src ="domain1.png" width="600">

From Photos to Emoji 

<img src ="domain1.png" width="600">

### SRGAN и StackGANs

https://arxiv.org/pdf/1609.04802v5.pdf

SRGAN (Super resolution GAN), StackGANs - сети для повышения разрешения изображений, отличающаяся от других сетей повышенным качеством восстановления мелких деталей и текстур изображения.

<img src ="SRGAN.jpg" width="700">

Сравнение с другими методами повышения разрешения:

<img src ="SRGAN_res.jpg" width="700">

### Pix2Pix

https://arxiv.org/abs/1611.07004v3

Pix2Pix - сети, переводящая пиксельные рисунки в реалистичные изображения. Она тоже использует принципы GAN для работы.

<img src ="pix2pix.png" width="500">

Попробовать сеть можно на сайте:

**https://affinelayer.com/pixsrv/**

**https://affinelayer.com/pix2pix/**

Ещё примеры:

<img src ="pix2pix2.jpg" width="500">
<img src ="pix2pix3.jpg" width="500">
<img src ="http://edunet.kea.su/repo/src/L13_GAN_cGAN/img/11-38.png" width="500">

### Семантическая генерация

Помимо шума, в модель можно подавать описание того что мы хотим получить. Это описание может быть разным, например текст или вектор заданных свойств.

Например:
* Класс объекта
* Углы и повороты
* Заданные параметры трансформаций
* Сегментация

<img src ="http://edunet.kea.su/repo/src/L13_GAN_cGAN/img/11-06.png" width="700">

Результат:

<img src ="http://edunet.kea.su/repo/src/L13_GAN_cGAN/img/11-07.png" width="300">

### Text to image

https://arxiv.org/abs/2001.06658

### ControlGAN

https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=8641270

https://github.com/mrlibw/ControlGAN

Описание может быть представлено в виде естественного текста. **ControlGAN** - Pytorch реализация  для управляемого преобразования текста в изображение. Цель состоит в том, чтобы генерировать изображения из текста, а также позволить пользователю манипулировать синтетическими изображениями, используя описания на естественном языке, в одной структуре.

<img src ="controlGAN.jpg" width="500">

Результаты:

<img src ="http://edunet.kea.su/repo/src/L13_GAN_cGAN/img/11-31.png" width="500">


Поэкспериментировать :

**https://deepai.org/machine-learning-model/text2img**

Одна из возможных архитектур с использованием RNN на входе:

<img src ="http://edunet.kea.su/repo/src/L13_GAN_cGAN/img/11-32.png" width="700">

# Заключение

Принципы генеративно-состязательных сетей вышли далеко за пределы генерации из шума. Сейчас с помощью GAN создаются сложнейшие state-of-art сети для самых разнообразных задач. В лекции были рассмотрены самые главные модели: GAN - для генерации из шума, DCGAN - для генерации изображений, с помомщью развёрток и cGAN - сети с генерацией по условию. 

Совмещая различные сети и подходы можно сконструировать множество самых разнообразных моделей.

# Ссылки

**cDCGAN**

https://github.com/znxlwm/pytorch-MNIST-CelebA-cGAN-cDCGAN

**StackGAN**

https://arxiv.org/pdf/1612.03242v2.pdf

https://arxiv.org/pdf/1710.10916.pdf

https://medium.com/@rangerscience/lets-read-science-stackgan-text-to-photo-realistic-image-synthesis-4562b2b14059

https://www.rulit.me/data/programs/resources/pdf/Generative-Adversarial-Networks-with-Python_RuLit_Me_610886.pdf

Видео:

https://www.youtube.com/watch?v=PXWIaLE7_NU

https://www.youtube.com/watch?v=crI5K4RCZws

https://github.com/zeusm9/Text-to-Photo-realistic-Image-Synthesis-with-Stacked-Generative-Adversarial-Networks

**ControlGAN**

https://github.com/mrlibw/ControlGAN

https://github.com/taki0112/ControlGAN-Tensorflow

https://arxiv.org/pdf/1708.00598.pdf

https://arxiv.org/pdf/1909.07083.pdf

https://arxiv.org/pdf/1910.05774.pdf

https://meta-guide.com/data/data-processing/text-to-image-systems/natural-language-text-to-image-2019

**AC-GAN**

https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html

https://machinelearningmastery.com/how-to-develop-an-auxiliary-classifier-gan-ac-gan-from-scratch-with-keras/

https://towardsdatascience.com/understanding-acgans-with-code-pytorch-2de35e05d3e4

https://arxiv.org/pdf/1909.05370.pdf

https://openaccess.thecvf.com/content/WACV2021/papers/Kavalerov_A_Multi-Class_Hinge_Loss_for_Conditional_GANs_WACV_2021_paper.pdf

https://github.com/lukedeo/keras-acgan